In [1]:
import numpy as np
import pandas as pd
import random
import torch
import pickle

In [2]:
from torch.utils.tensorboard import SummaryWriter

In [3]:
from PortfolioConstructor import PortfolioConstructor
from ExchnageEnv import MarketEnvironment

In [4]:
if torch.cuda.is_available():
    torch.set_default_device('cuda') 
    torch.get_default_device()
    device = 'cuda'
    
else:
    device = 'cpu'

print(f"device : {device}")

device : cuda


In [5]:
pkl_fpath = '/home/naradaw/dev/Charles_Schwab/data/historical_random_100/2024_10_15/historical_price_seq_2024_10_15_16_22.pkl'

with open(pkl_fpath, 'rb') as f:
    price_sqs_dict = pickle.load(f)

In [6]:
price_sqs_dict[list(price_sqs_dict.keys())[0]].shape

(1174, 61)

In [7]:
symbol_universe = list(price_sqs_dict.keys())
len(symbol_universe)

100

In [8]:
# test_symbol_uni = random.choices(list(price_sqs_dict.keys()), k = 20)


In [9]:
portfolio_constructor = PortfolioConstructor(
    device= device,
    symbol_universe = symbol_universe,
    seq_length = 60,
    multihead_dim = 2,
    num_transformer_layers = 2
)

/home/naradaw/miniconda3/envs/tf-wsl/lib/python3.9/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [10]:
market_env = MarketEnvironment(
    data_path = pkl_fpath,
    holding_period = 1,
    train_test_split= 0.8,
    symbol_universe = symbol_universe,
    device = device
    )

In [11]:
market_env.data.shape

(1174, 100, 61)

# Utility

In [12]:
''' 
sharpe ratio measures the excess return of the portfolio over the 
volatility of it -> risk adjusted performance
'''


def sharp_ratio_(rewards, tran_costs):

	# rewards = [r.detach().cpu().numpy() for r in rewards]
	mean = sum(rewards) / len(rewards)
	At = sum(r - t for r, t in zip(rewards, tran_costs)) / len(rewards)
	vol = sum((r - mean) ** 2 for r in rewards) / len(rewards)
	vol = vol ** 0.5

	return (At - 1e-7) / (vol + 1e-9)

# Train

In [13]:
portfolio_constructor.cuda()
portfolio_constructor.train()
market_env.reset(mode = "train")

In [14]:
portfolio_constructor.parameters()

<generator object Module.parameters at 0x7f85481347b0>

In [15]:
optimizer = torch.optim.Adam(portfolio_constructor.parameters())

In [16]:
market_env.get_state()

tensor([[121.5000, 120.6800, 116.4500,  ..., 129.1800, 129.3300, 130.3400],
        [117.2400, 116.6700, 116.9600,  ..., 141.1400, 139.4600, 139.6500],
        [147.6800, 148.4200, 141.9300,  ..., 152.9100, 154.4500, 154.4600],
        ...,
        [ 44.2800,  43.9700,  44.2900,  ...,  45.9100,  46.0100,  46.0700],
        [ 73.6800,  73.8900,  73.2100,  ...,  85.6100,  86.2300,  85.4600],
        [105.9000, 107.6400, 107.3000,  ..., 132.8500, 133.3600, 133.4900]],
       device='cuda:0')

# Sandbox

In [17]:
def sharp_ratio_loss_(rewards, tran_costs, allocations):

	# rewards = [r.detach().cpu().numpy() for r in rewards]
	mean = sum(rewards) / len(rewards)
	At = sum(r - t for r, t in zip(rewards, tran_costs)) / len(rewards)
	vol = sum((r - mean) ** 2 for r in rewards) / len(rewards)
	vol = vol ** 0.5

	return (At - 1e-7) / (vol + 1e-9)

In [18]:
training_steps = 500
eval_step = 32
train_step = 32


In [19]:
def evaluate(model, env):
    model.eval()
    is_end = False
    rewards = []
    tran_costs = []
    
    env.reset(mode = "test")
    state = env.get_state()

    while not is_end:
        _ , allocations = model(state)
        state, reward, is_end, tran_cost = env.step(allocations)

        rewards.append(reward)
        tran_costs.append(tran_cost)

    sharp_ratio = sharp_ratio_(rewards, tran_costs)
    
    model.train()

    return sharp_ratio

In [20]:
writer = SummaryWriter()

In [21]:
max_reward = -1

for training_step in range(training_steps):
    is_end = False
    rewards = []
    tran_costs = []
    nlls = []

    market_env.reset(mode = "train", transaction_cost= 1e-7)
    state = market_env.get_state()

    while not is_end:
        symbol_idx, allocations = portfolio_constructor(state)
        state, reward, is_end, tran_cost = market_env.step(allocations)

        rewards.append(reward)
        tran_costs.append(tran_cost)
        mask_tensor = torch.tensor([1 if i in symbol_idx.cpu().numpy() else 0 for i in range(allocations.shape[0])]).type(torch.FloatTensor).cuda()
        nlls.append(torch.log(allocations.abs() / 2 + 1e-9) * mask_tensor)
        # nlls.append(-(torch.log(allocations.abs() / 2 + 1e-9) * mask_tensor))

    
    sharp_ratio = sharp_ratio_(rewards, tran_costs)
    loss = -sharp_ratio * sum([e.sum() for e in nlls])

    loss.backward(retain_graph=True)

    if (training_step + 1) % train_step == 0:

        print("-------------------------------------")
        print("training model --")
        print('Step {}: last loss = {:.5f}\r'.format(training_step, loss), end='')
        print()
        writer.add_scalar("Loss/train", sharp_ratio, training_step)
        # pprint([(n, e.grad) for n, e in model.named_parameters()])
        optimizer.step()
        optimizer.zero_grad()
        count = 0
        
    if (training_step + 1) % eval_step == 0:
        print("eval step --")
        with torch.no_grad():
            reward_val = evaluate(portfolio_constructor, market_env)
            print('Step {}: val_rewards = {}'.format(training_step, reward_val))
            writer.add_scalar("eval_sharpe/train", reward_val, training_step)

            if max_reward < reward_val:
                max_reward = reward_val

                print("*** found better model ***")
            print()
                # torch.save(portfolio_constructor.state_dict(), model_path)

-------------------------------------
training model --
Step 31: last loss = 45.61536
eval step --
Step 31: val_rewards = 0.4591165858777075
*** found better model ***

-------------------------------------
training model --
Step 63: last loss = 24.92854
eval step --
Step 63: val_rewards = 0.11485735874186744

-------------------------------------
training model --
Step 95: last loss = 89.21492
eval step --
Step 95: val_rewards = -0.05456413280536202

-------------------------------------
training model --
Step 127: last loss = 101.86233
eval step --
Step 127: val_rewards = -0.1003365504610342

-------------------------------------
training model --
Step 159: last loss = 250.56691
eval step --
Step 159: val_rewards = -0.08374083424651758

-------------------------------------
training model --
Step 191: last loss = 75.63830
eval step --
Step 191: val_rewards = -0.17835616320479927

-------------------------------------
training model --
Step 223: last loss = 133.93294
eval step --
Step

In [22]:
random_state = market_env.get_random_state()
random_state.detach().cpu().numpy()

array([[140.85, 143.62, 143.96, ..., 132.63, 130.35, 129.18],
       [239.24, 242.92, 243.09, ..., 252.98, 259.6 , 258.78],
       [120.1 , 122.41, 130.58, ...,  99.82,  95.83,  95.08],
       ...,
       [ 40.47,  40.15,  39.78, ...,  39.74,  39.55,  39.95],
       [ 79.5 ,  78.76,  79.79, ...,  77.43,  77.73,  77.74],
       [205.58, 205.42, 205.19, ..., 211.91, 211.91, 212.57]],
      dtype=float32)

In [23]:
portfolio_constructor(random_state)

(tensor([ 4, 68, 70,  6, 44, 13, 49, 18, 28, 95], device='cuda:0'),
 tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0991, 0.0000, 0.0998, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.1005, 0.0000, 0.0000, 0.0000, 0.0000,
         0.1022, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0999, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0991,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0996, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1006, 0.0000, 0.0995, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0998, 0.0000, 0.0000, 0.0000,
         0.0000], device='cuda:0', grad_f

In [24]:
writer.add_graph(portfolio_constructor, random_state.detach())

/home/naradaw/dev/Charles_Schwab/code/RnD/enviroment/PortfolioConstructor.py:142: TracerWarning: torch.Tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  long_mask = torch.Tensor([0 if i in long_sqs else 1 for i in range(rank.shape[0])]).to(self.device)
/home/naradaw/dev/Charles_Schwab/code/RnD/enviroment/PortfolioConstructor.py:170: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  portfolio_allocations = [allocation.item() for allocation in allocations if allocation != 0]


In [25]:
writer.flush()
writer.close()

In [26]:
!tensorboard --logdir=runs

TensorFlow installation not found - running with reduced feature set.

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.18.0 at http://localhost:6006/ (Press CTRL+C to quit)
^C


In [33]:
torch.log(torch.tensor([0.4, 0.3, 0.3])).sum(), torch.log(torch.tensor([0.8, 0.1, 0.1])).sum()

(tensor(-3.3242, device='cuda:0'), tensor(-4.8283, device='cuda:0'))